In [2]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Set some display options for pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# Load the processed data from the CSV file
file_path = '../data/processed/cleaned_purchase_orders.csv'
df = pd.read_csv(file_path)

# Convert created_date back to datetime object for time-series analysis
df['created_date'] = pd.to_datetime(df['created_date'])

In [5]:
# -----------------------------
# Section A: Performance Overview KPIs
# -----------------------------
required_cols = [
'purchasing_group', 'plant', 'material_group', 'description', 'supplier_id',
'product_id', 'quantity', 'unit_price', 'net_value', 'month', 'unit' , 'purchase_order_id' , 'created_date' , 'status'
]


for col in required_cols:
    if col not in df.columns:
        print(f"Warning: Column '{col}' not found in dataset")


# KPIs
kpi_total_spend = df['net_value'].sum()
kpi_total_quantity = df['quantity'].sum()
total_skus = df['product_id'].nunique()


# Fragmentation analysis
supplier_counts_per_sku = df.groupby('product_id')['supplier_id'].nunique()
kpi_fragmented_skus = (supplier_counts_per_sku > 1).sum()
kpi_fragmentation_rate = kpi_fragmented_skus / total_skus if total_skus > 0 else 0

# Saving Potential Calculation
min_prices = df.groupby('product_id')['unit_price'].min().reset_index()
min_prices.rename(columns={'unit_price': 'min_price'}, inplace=True)
df_merged = pd.merge(df, min_prices, on='product_id')
df_merged['potential_saving'] = (df_merged['unit_price'] - df_merged['min_price']) * df_merged['quantity']
kpi_saving_potential = df_merged['potential_saving'].sum()

# --- Additional KPIs ---

# 1. Supplier-Related KPIs
kpi_active_suppliers = df['supplier_id'].nunique()
top_5_spend = df.groupby('supplier_id')['net_value'].sum().nlargest(5).sum()
kpi_top_5_spend_pct = (top_5_spend / kpi_total_spend) * 100 if kpi_total_spend > 0 else 0
kpi_single_sourced_skus = (supplier_counts_per_sku == 1).sum()

# 2. Operational & Process KPIs
kpi_total_pos = df['purchase_order_id'].nunique()
po_values = df.groupby('purchase_order_id')['net_value'].sum()
kpi_avg_po_value = po_values.mean()
line_items_per_po = df.groupby('purchase_order_id').size()
kpi_avg_line_items = line_items_per_po.mean()
pending_pos = df[df['status'] == 'Pending']['purchase_order_id'].nunique()
kpi_pending_rate_pct = (pending_pos / kpi_total_pos) * 100 if kpi_total_pos > 0 else 0


In [6]:
# -----------------------------
# Section B: Analysis & Trends
# -----------------------------
# Spend trend by month
df['created_date'] = pd.to_datetime(df['created_date'])

# Group by month using the actual date, then format the label
spend_trend = df.groupby(df['created_date'].dt.to_period('M')).agg({
    'net_value':'sum',
    'quantity':'sum'
}).reset_index()

# Sort chronologically
spend_trend = spend_trend.sort_values('created_date')

# Convert the period to a more readable string format for the final report
spend_trend['month'] = spend_trend['created_date'].dt.strftime('%Y-%m')

print("Correctly sorted Spend Trend:")
print(spend_trend)



# Spend by purchasing_group
purchasing_group_spend = df.groupby('purchasing_group').agg({
    'net_value':'sum',
    'quantity':'sum'
}).reset_index().sort_values('net_value', ascending=False)


# Top SKUs by spend
top_skus = df.groupby(['product_id','description']).agg({
    'net_value':'sum',
    'quantity':'sum'
}).reset_index().sort_values('net_value', ascending=False).head(10)


# Supplier price analysis (avg unit prices)
# A more meaningful supplier analysis

# First, get the list of your top 10 SKUs by spend (from your previous step)
top_10_sku_list = top_skus['product_id'].tolist()

# Filter the main dataframe to only include these top SKUs
top_skus_df = df[df['product_id'].isin(top_10_sku_list)]

# Now, find the best price offered for each of these SKUs
best_prices_per_sku = top_skus_df.groupby('product_id')['unit_price'].min().reset_index()
best_prices_per_sku.rename(columns={'unit_price': 'best_price'}, inplace=True)

# Merge this back to find which supplier offered that best price
supplier_scorecard = pd.merge(
    best_prices_per_sku,
    top_skus_df[['product_id', 'supplier_id', 'unit_price']],
    left_on=['product_id', 'best_price'],
    right_on=['product_id', 'unit_price']
)

# Clean up and show the final scorecard
supplier_scorecard = supplier_scorecard[['product_id', 'supplier_id', 'best_price']].drop_duplicates()

print("\nSupplier Scorecard (Who is cheapest for our Top SKUs):")
print(supplier_scorecard)


# Spend by Plant
plant_spend = df.groupby('plant').agg({
    'net_value':'sum',
    'quantity':'sum'
}).reset_index().sort_values('net_value', ascending=False)

print(plant_spend)

# Spend by Material Group
material_group_spend = df.groupby('material_group').agg({
    'net_value':'sum',
    'quantity':'sum'
}).reset_index().sort_values('net_value', ascending=False)

print(material_group_spend)


Correctly sorted Spend Trend:
   created_date  net_value  quantity    month
0       2025-01  107527.67  75573.00  2025-01
1       2025-02  110545.33  80023.00  2025-02
2       2025-03  312557.03 142140.00  2025-03
3       2025-04   34987.49  15358.00  2025-04
4       2025-05  231804.23 135536.00  2025-05
5       2025-06  143897.72  94935.00  2025-06
6       2025-07  248165.67 137692.00  2025-07
7       2025-08  168856.99 115094.00  2025-08
8       2025-09   63305.44  33519.00  2025-09
9       2025-10  119131.15  69936.00  2025-10
10      2025-11  117691.03  81776.00  2025-11
11      2025-12   77346.41  49062.00  2025-12

Supplier Scorecard (Who is cheapest for our Top SKUs):
   product_id   supplier_id  best_price
0    MAT-3159  SUPP_PACK_08        0.60
1    MAT-4077  SUPP_PACK_06        3.01
2    MAT-4862  SUPP_PACK_03        1.80
4    MAT-5204  SUPP_PACK_07        1.21
5    MAT-5204  SUPP_PACK_02        1.21
7    MAT-5627  SUPP_PACK_02        0.83
8    MAT-5627  SUPP_PACK_03        0

In [7]:
# -----------------------------
# Section C: Procurement Recommendations (Revised)
# -----------------------------

# 1. Create a summary table for each SKU
sku_summary = df.groupby(['product_id', 'description']).agg(
    total_quantity=('quantity', 'sum'),
    total_net_value=('net_value', 'sum'),
    avg_unit_price=('unit_price', 'mean'),
    purchase_frequency_months=('month', 'nunique'),
    supplier_count=('supplier_id', 'nunique')
).reset_index().sort_values('total_net_value', ascending=False)

# Add a cumulative spend column to identify top items
sku_summary['cum_spend_pct'] = (sku_summary['total_net_value'].cumsum() / sku_summary['total_net_value'].sum()) * 100

# 2. Identify high-value, recurring, fragmented SKUs as Contract Candidates
contract_candidates = sku_summary[
    (sku_summary['purchase_frequency_months'] >= 3) &
    (sku_summary['supplier_count'] > 1) &
    (sku_summary['cum_spend_pct'] <= 80) # <-- Focus on top 80% spend
].copy() # Using .copy() to avoid SettingWithCopyWarning

# 3. Find the best (minimum) price for every SKU from the original data
best_suppliers = df.loc[df.groupby('product_id')['unit_price'].idxmin()]
best_suppliers = best_suppliers[['product_id', 'supplier_id', 'unit_price']]
best_suppliers.rename(columns={'supplier_id': 'Recommended Supplier', 'unit_price': 'Best Price'}, inplace=True)

# 4. Generate recommendations by merging candidates with best suppliers (NO LOOP)
recommendations_df = pd.merge(contract_candidates, best_suppliers, on='product_id')

# 5. Calculate estimated savings
recommendations_df['Estimated Saving'] = (recommendations_df['avg_unit_price'] - recommendations_df['Best Price']) * recommendations_df['total_quantity']

# 6. Clean up the final recommendations table
recommendations_df = recommendations_df[[
    'product_id', 'description', 'Recommended Supplier', 'Best Price',
    'avg_unit_price', 'total_quantity', 'Estimated Saving', 'purchase_frequency_months', 'supplier_count'
]].sort_values('Estimated Saving', ascending=False)

print("\nTop Recommendations:")
print(recommendations_df.head())



Top Recommendations:
  product_id          description Recommended Supplier  Best Price  \
0   MAT-6002         Soybean Meal         SUPP_PACK_02        2.00   
2   MAT-4077  Rumen Protected Fat         SUPP_PACK_06        3.01   
1   MAT-5204                 Corn         SUPP_PACK_07        1.21   

   avg_unit_price  total_quantity  Estimated Saving  \
0            2.23       281605.00          63727.21   
2            3.80        68252.00          54028.28   
1            1.38       222894.00          38777.84   

   purchase_frequency_months  supplier_count  
0                         10               9  
2                          8               7  
1                         10               8  


In [8]:
# -----------------------------
# Section D: SKU Analysis Table (Enhanced)
# -----------------------------

# 1. Your original, correct aggregation
sku_analysis = df.groupby(['product_id', 'description', 'purchasing_group', 'supplier_id']).agg(
    quantity_purchased=('quantity', 'sum'),
    net_value_spent=('net_value', 'sum'),
    avg_price_paid=('unit_price', 'mean')
).reset_index()

# 2. Find the total quantity purchased for each SKU to calculate contribution
sku_total_quantity = df.groupby('product_id')['quantity'].sum().reset_index()
sku_total_quantity.rename(columns={'quantity': 'total_sku_quantity'}, inplace=True)

# 3. Find the best (minimum) price available for each SKU across all suppliers
best_prices = df.groupby('product_id')['unit_price'].min().reset_index()
best_prices.rename(columns={'unit_price': 'best_available_price'}, inplace=True)

# 4. Merge these new context columns into the main analysis table
sku_analysis = pd.merge(sku_analysis, sku_total_quantity, on='product_id')
sku_analysis = pd.merge(sku_analysis, best_prices, on='product_id')

# 5. Calculate the new analysis columns
sku_analysis['contribution_to_total_qty_%'] = (sku_analysis['quantity_purchased'] / sku_analysis['total_sku_quantity']) * 100
sku_analysis['price_variance'] = sku_analysis['avg_price_paid'] - sku_analysis['best_available_price']
sku_analysis['cost_above_best_price'] = sku_analysis['price_variance'] * sku_analysis['quantity_purchased']

# 6. Clean up and sort the final table to show the largest cost variances first
sku_analysis = sku_analysis[[
    'purchasing_group', 'product_id', 'description', 'supplier_id',
    'quantity_purchased', 'contribution_to_total_qty_%',
    'avg_price_paid', 'best_available_price', 'price_variance', 'cost_above_best_price'
]].sort_values('cost_above_best_price', ascending=False)

print("\nEnhanced SKU Analysis Table (showing highest overspending first):")
print(sku_analysis.head())



Enhanced SKU Analysis Table (showing highest overspending first):
    purchasing_group product_id          description   supplier_id  \
100          PG-MEAL   MAT-6002         Soybean Meal  SUPP_PACK_03   
98           PG-MEAL   MAT-6002         Soybean Meal  SUPP_PACK_01   
26           PG-MEAL   MAT-4077  Rumen Protected Fat  SUPP_PACK_01   
101          PG-MEAL   MAT-6002         Soybean Meal  SUPP_PACK_08   
57          PG-GRAIN   MAT-5204                 Corn  SUPP_PACK_01   

     quantity_purchased  contribution_to_total_qty_%  avg_price_paid  \
100            71020.00                        25.22            2.22   
98             52822.00                        18.76            2.23   
26             14865.00                        21.78            3.74   
101            45671.00                        16.22            2.23   
57             48913.00                        21.94            1.42   

     best_available_price  price_variance  cost_above_best_price  
100         

In [9]:
# ----------------------------------------------------------------------
# Section E: Supplier Performance & Risk Management
# ----------------------------------------------------------------------

# 1. Supplier Concentration Risk (for High-Value Items)
# We'll identify single-sourced SKUs that are in the top 80% of spend.
top_spend_skus = sku_summary[sku_summary['cum_spend_pct'] <= 80]['product_id']
single_sourced_mask = sku_summary['product_id'].isin(top_spend_skus) & (sku_summary['supplier_count'] == 1)
critical_single_sourced_skus = sku_summary[single_sourced_mask]

# Merge with supplier info to show who the single supplier is
supplier_info = df[['product_id', 'supplier_id']].drop_duplicates()
critical_risk_suppliers = pd.merge(critical_single_sourced_skus, supplier_info, on='product_id')

# Clean up the final table
critical_risk_suppliers = critical_risk_suppliers[[
    'product_id', 'description', 'supplier_id', 'total_net_value'
]].sort_values('total_net_value', ascending=False)


# 2. Price Volatility Analysis
# We use the Coefficient of Variation (CV = std dev / mean) to measure volatility.
# A higher CV means more unstable prices.
price_volatility = df.groupby(['product_id', 'description'])['unit_price'].agg(['mean', 'std']).reset_index()
price_volatility['CV_%'] = (price_volatility['std'] / price_volatility['mean']) * 100
price_volatility = price_volatility.sort_values('CV_%', ascending=False).fillna(0)


# Export section for the new dataframes
# with pd.ExcelWriter("procurement_dashboard_final.xlsx", mode='a', engine='openpyxl') as writer:
#     critical_risk_suppliers.to_excel(writer, sheet_name="E_Critical_Supplier_Risk", index=False)
#     price_volatility.to_excel(writer, sheet_name="E_Price_Volatility", index=False)

print("\n--- Section E: Supplier Risk Analysis ---")
print("\nCritical Single-Sourced SKUs (High Value, High Risk):")
print(critical_risk_suppliers.head())

print("\nTop 5 Most Volatile SKUs by Price:")
print(price_volatility.head())



--- Section E: Supplier Risk Analysis ---

Critical Single-Sourced SKUs (High Value, High Risk):
Empty DataFrame
Columns: [product_id, description, supplier_id, total_net_value]
Index: []

Top 5 Most Volatile SKUs by Price:
  product_id          description  mean  std  CV_%
6   MAT-6214           Wheat Bran  0.72 0.12 17.18
0   MAT-3159             Molasses  0.84 0.12 14.50
1   MAT-4077  Rumen Protected Fat  3.80 0.44 11.56
4   MAT-5627              Alfalfa  0.97 0.09  9.83
7   MAT-7729         Barley Grain  1.21 0.11  9.10


In [13]:
# You may need to install this library: pip install statsmodels
from statsmodels.tsa.api import SimpleExpSmoothing

# ----------------------------------------------------------------------
# Section F: Demand Forecasting
# ----------------------------------------------------------------------

# We will forecast demand for the Top 5 SKUs by quantity
top_5_skus_by_qty = df.groupby('product_id')['quantity'].sum().nlargest(5).index.tolist()

# 1. Prepare the monthly time-series data for these SKUs
monthly_demand = df[df['product_id'].isin(top_5_skus_by_qty)].copy()
monthly_demand['created_date'] = pd.to_datetime(monthly_demand['created_date'])
monthly_demand_ts = monthly_demand.groupby(['product_id', pd.Grouper(key='created_date', freq='M')])['quantity'].sum().reset_index()

# 2. Loop through each top SKU and generate a forecast
all_forecasts = []
for sku in top_5_skus_by_qty:
    # Get the historical data for this one SKU
    sku_history = monthly_demand_ts[monthly_demand_ts['product_id'] == sku].set_index('created_date')['quantity']
    
    # If there's enough data, create and fit the model
    if len(sku_history) > 2:
        # Fit the Simple Exponential Smoothing model
        model = SimpleExpSmoothing(sku_history, initialization_method="estimated").fit()
        # Forecast the next 3 months
        forecast = model.forecast(3).rename(f"{sku}_forecast").reset_index()
        forecast.columns = ['Forecast_Date', 'Forecast_Quantity']
        forecast['product_id'] = sku
        all_forecasts.append(forecast)

# 3. Combine all forecasts into a single DataFrame
if all_forecasts:
    demand_forecast_df = pd.concat(all_forecasts)
    demand_forecast_df = demand_forecast_df[['product_id', 'Forecast_Date', 'Forecast_Quantity']]
    
    print("\n--- Section F: Demand Forecasting ---")
    print("\n3-Month Demand Forecast for Top 5 SKUs:")
    print(demand_forecast_df)

    # Export section for the new dataframe
    # with pd.ExcelWriter("procurement_dashboard_final.xlsx", mode='a', engine='openpyxl') as writer:
    #     demand_forecast_df.to_excel(writer, sheet_name="F_Demand_Forecast", index=False)


--- Section F: Demand Forecasting ---

3-Month Demand Forecast for Top 5 SKUs:
  product_id        Forecast_Date  Forecast_Quantity
0   MAT-6002  2026-01-31 00:00:00            8185.97
1   MAT-6002  2026-02-28 00:00:00            8185.97
2   MAT-6002  2026-03-31 00:00:00            8185.97
0   MAT-5204                   10            5566.00
1   MAT-5204                   11            5566.00
2   MAT-5204                   12            5566.00
0   MAT-4862                    7           16071.44
1   MAT-4862                    8           16071.44
2   MAT-4862                    9           16071.44
0   MAT-6214  2026-01-31 00:00:00           13507.35
1   MAT-6214  2026-02-28 00:00:00           13507.35
2   MAT-6214  2026-03-31 00:00:00           13507.35
0   MAT-3159  2026-01-31 00:00:00           10538.20
1   MAT-3159  2026-02-28 00:00:00           10538.20
2   MAT-3159  2026-03-31 00:00:00           10538.20


C:\Users\Nazel\AppData\Local\Temp\ipykernel_18396\2049901187.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_demand_ts = monthly_demand.groupby(['product_id', pd.Grouper(key='created_date', freq='M')])['quantity'].sum().reset_index()
C:\Users\Nazel\OneDrive\Desktop\nadec-analysis\procurement_sku_analysis\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
C:\Users\Nazel\OneDrive\Desktop\nadec-analysis\procurement_sku_analysis\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Nazel\OneDrive\Desktop\nadec-analysis\procurement_sku_analysis\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: V

  Using cached statsmodels-0.14.5-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached scipy-1.16.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached statsmodels-0.14.5-cp313-cp313-win_amd64.whl (9.6 MB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
Using cached scipy-1.16.2-cp313-cp313-win_amd64.whl (38.5 MB)

   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   ---------------------------------------- 0/3 [scipy]
   -------------------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
print("hello")

hello
